In [2]:
import numpy as np
import librosa
import os
from matplotlib import pyplot as plt
!pip install pretty_midi
import pretty_midi 
pretty_midi.pretty_midi.MAX_TICK = 1e10

     |████████████████████████████████| 5.6MB 1.5MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
  Created wheel for pretty-midi: filename=pretty_midi-0.2.9-cp37-none-any.whl size=5591958 sha256=f95fdb8f56cbd70943aef58132ebb395a04d857fe5e233d95ce659566220a66e
  Stored in directory: /root/.cache/pip/wheels/4c/a1/c6/b5697841db1112c6e5866d75a6b6bf1bef73b874782556ba66
Successfully built pretty-midi


In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#preparing the data from the original audio files
entries = os.scandir('/content/drive/MyDrive/train_source')#loading the data to process which we are going to use for training 
os.mkdir('/content/drive/MyDrive/trainfiles')
os.mkdir('/content/drive/MyDrive/trainfiles/trainwav')
os.mkdir('/content/drive/MyDrive/trainfiles/trainmid')

for file in entries:
    file_name,file_extensions=os.path.splitext(file)

    if file_extensions=='.wav':
        y, sr = librosa.load(file)
        sr=16000
        cqt_file=librosa.cqt(y, sr=16000,fmin=None, n_bins=252, bins_per_octave=36)#using the librosa library to calculate the cqt of the audio file which is returned as a 2D vector 
        c=len(y) # it is being stored because it will be used for the frequency while making the piano roll file
        cqt=np.abs(cqt_file)# the returned 2d vector is a complex quantity 
        cqt=librosa.amplitude_to_db(cqt)# taking the amplitude on logarithmic scale gives a better visualization of data
        b=np.shape(cqt)[1]
        out = '/content/drive/MyDrive/trainfiles/trainwav/wav_'+str(file_name[40:])+'.npy'
        print(np.shape(cqt))
        np.save(out,cqt)
        print('wrote file', out)
        entries1 = os.scandir('/content/drive/MyDrive/train_source')#it is being loaded again so that we can get the corresponding midi file to the wav file which we have just used

        for file1 in entries1:
          file_name1,file_extensions1=os.path.splitext(file1)

          if file_name1==file_name and file_extensions1==".mid":
            file2=file_name + file_extensions1
            pm = pretty_midi.PrettyMIDI(file2)
            a=pm.instruments[0].get_piano_roll(float(sr*float(b)/float(c)))# The Sampling frequency has been taken such that its shape and the cqt files shape is same 
            a=np.append(np.transpose(a), np.zeros((b-np.shape(a)[1],128)), axis=0)# there is a slight difference between the length of the cqt data and midi data , to make the number of features same for training a padding of few row is added
            a=np.transpose(a)
            print(np.shape(a))
            out = '/content/drive/MyDrive/trainfiles/trainmid/mid_'+str(file_name1[40:])+'mid'+'.npy'
            np.save(out,a)
            print("saving", out)

  

(252, 10292)
wrote file /content/drive/MyDrive/trainfiles/trainwav/wav__MUS-alb_se2_StbgTGd2.npy
(128, 10292)
saving /content/drive/MyDrive/trainfiles/trainmid/mid__MUS-alb_se2_StbgTGd2mid.npy
(252, 16618)
wrote file /content/drive/MyDrive/trainfiles/trainwav/wav__MUS-bk_xmas1_StbgTGd2.npy
(128, 16618)
saving /content/drive/MyDrive/trainfiles/trainmid/mid__MUS-bk_xmas1_StbgTGd2mid.npy
(252, 9943)
wrote file /content/drive/MyDrive/trainfiles/trainwav/wav__MUS-bk_xmas4_StbgTGd2.npy
(128, 9943)
saving /content/drive/MyDrive/trainfiles/trainmid/mid__MUS-bk_xmas4_StbgTGd2mid.npy
(252, 17047)
wrote file /content/drive/MyDrive/trainfiles/trainwav/wav__MUS-bk_xmas5_StbgTGd2.npy
(128, 17047)
saving /content/drive/MyDrive/trainfiles/trainmid/mid__MUS-bk_xmas5_StbgTGd2mid.npy
(252, 4955)
wrote file /content/drive/MyDrive/trainfiles/trainwav/wav__MUS-bor_ps6_StbgTGd2.npy
(128, 4955)
saving /content/drive/MyDrive/trainfiles/trainmid/mid__MUS-bor_ps6_StbgTGd2mid.npy
(252, 4866)
wrote file /content/d

In [ ]:
path1='/content/drive/MyDrive/trainfiles/trainwav'

def concatenate_wav(path):#we are dividing the number of features of the cqt file into batches of 100 and stacking them up to create a 3D array 
  entries = os.scandir(path)
  d=np.zeros((1,252,100))#initialising the array with the correct shape 

  for file in entries:
      a=np.load(file)
      a=a[np.newaxis,:,:]# adding a new axis to the original files
      for b in range(int(np.shape(a)[2]/100)):
            
          c=a[:,:,b*100:(b*100)+100]# dividing them into batches of 100
          d=np.concatenate((d,c), axis=0)# stacking them up on the first axis 

  out = '/content/drive/MyDrive/trainfiles/trainwav/concatenate'+'wav'+'.npy'
  d=d[1:,:,:]# array was initialise with zeros so excluding that part 
  d=np.transpose(d,(0,2,1))# taking transpose so that the the second and third dimensions are interchanged
  np.save(out,d)  

concatenate_wav(path1)

path='/content/drive/MyDrive/trainfiles/trainmid'

def concatenate_mid(path):#we are dividing the number of features of the pianoroll file into batches of 100 and stacking them up to create a 3D array 
  entries = os.scandir(path)
  d=np.zeros((1,88,100))

  for file in entries:
      a=np.load(file)
      a=a[np.newaxis,20:108,:]
      for b in range(int(np.shape(a)[2]/100)):
            
          c=a[:,:,b*100:(b*100)+100]
          d=np.concatenate((d,c), axis=0)
          
  out = '/content/drive/MyDrive/trainfiles/trainmid/concatenate'+'mid'+'.npy'
  d=d[1:,:,:]
  d=np.transpose(d,(0,2,1))
  np.save(out,d)  

concatenate_mid(path)


In [ ]:
#for the test files
entries = os.scandir('/content/drive/MyDrive/test_source')
os.mkdir('/content/drive/MyDrive/testfiles')
os.mkdir('/content/drive/MyDrive/testfiles/testwav')
os.mkdir('/content/drive/MyDrive/testfiles/testmid')

for file in entries:
    file_name,file_extensions=os.path.splitext(file)
    if file_extensions=='.wav':
        y, sr = librosa.load(file)
        n_fft=2048
        hop_length=512
        sr=16000
        cqt_file=librosa.cqt(y, sr=16000,fmin=None, n_bins=252, bins_per_octave=36)
        c=len(y)
        cqt=np.abs(cqt_file)
        cqt=librosa.amplitude_to_db(cqt)
        b=np.shape(cqt)[1]
        out = '/content/drive/MyDrive/testfiles/testwav/wav_'+str(file_name[40:])+'.npy'
        np.save(out,cqt)
        entries1 = os.scandir('/content/drive/MyDrive/test_source')
        for file1 in entries1:
          file_name1,file_extensions1=os.path.splitext(file1)

          if file_name1==file_name and file_extensions1==".mid":
            file2=file_name + file_extensions1
            pm = pretty_midi.PrettyMIDI(file2)
            a=pm.instruments[0].get_piano_roll(float(sr*float(b)/float(c)))
            a=np.append(np.transpose(a), np.zeros((b-np.shape(a)[1],128)), axis=0)
            a=np.transpose(a)
            out = '/content/drive/MyDrive/testfiles/testmid/mid_'+str(file_name1[40:])+'mid'+'.npy'
            np.save(out,a)


In [ ]:
path='/content/drive/MyDrive/testfiles/testwav'

def concatenate_wavtest(path):
  entries = os.scandir(path)
  d=np.zeros((1,252,100))
  for file in entries:
      a=np.load(file)
      f=f+np.shape(a)[1]
      a=a[np.newaxis,:,:]
      for b in range(int(np.shape(a)[2]/100)):
            
          c=a[:,:,b*100:(b*100)+100]
          d=np.concatenate((d,c), axis=0)
  out = '/content/drive/MyDrive/testfiles/testwav/concatenate'+'wav'+'.npy'
  d=d[1:,:,:]
  d=np.transpose(d,(0,2,1))
  np.save(out,d)  

concatenate_wavtest(path)

path='/content/drive/MyDrive/testfiles/testmid'

def concatenate_midtest(path):
  entries = os.scandir(path)
  f=0
  d=np.zeros((1,88,100))
  for file in entries:      
      a=np.load(file)
      a=a[np.newaxis,20:108,:]
      for b in range(int(np.shape(a)[2]/100)):
            
          c=a[:,:,b*100:(b*100)+100]
          d=np.concatenate((d,c), axis=0)
  out = '/content/drive/MyDrive/testfiles/testmid/concatenate'+'mid'+'.npy'
  d=d[1:,:,:]
  d=np.transpose(d,(0,2,1))
  np.save(out,d)  

concatenate_midtest(path)